# imports and setup

In [96]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [97]:
# dagshub setup
import dagshub

dagshub.init(repo_owner='lmamu21', repo_name='House-Prices', mlflow=True)

Initialized MLflow to track repo "lmamu21/House-Prices"

Repository lmamu21/House-Prices initialized!

In [98]:
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = 'lmamu21' 
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd392b86afa2c37911a6814230b474c4b5df06fcb'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/lmamu21/House-Prices.mlflow'

In [99]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/lmamu21/House-Prices.mlflow")
mlflow.set_experiment("House-Prices")

<Experiment: artifact_location='mlflow-artifacts:/f9ed4fdc67b34e6b8b9d059aa67ad3d5', creation_time=1744051643876, experiment_id='3', last_update_time=1744051643876, lifecycle_stage='active', name='House-Prices', tags={}>

In [100]:
test = pd.read_csv("../data/test.csv")
train = pd.read_csv("../data/train.csv")
sample = pd.read_csv("../data/sample_submission.csv")


In [101]:
X = train.drop(columns=["SalePrice"])
y = train["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# data cleaning

### NA values

In [102]:
with pd.option_context('display.max_rows', None):
    print(X_train.isna().mean()[X_train.isna().mean() > 0])

LotFrontage     0.185788
Alley           0.936644
MasVnrType      0.584760
MasVnrArea      0.005137
BsmtQual        0.023973
BsmtCond        0.023973
BsmtExposure    0.023973
BsmtFinType1    0.023973
BsmtFinType2    0.023973
Electrical      0.000856
FireplaceQu     0.468322
GarageType      0.054795
GarageYrBlt     0.054795
GarageFinish    0.054795
GarageQual      0.054795
GarageCond      0.054795
PoolQC          0.994863
Fence           0.800514
MiscFeature     0.960616
dtype: float64


In [103]:
print(X_train['PoolArea'].value_counts())
print(X_train['PoolQC'].value_counts())
print(X_train['PoolQC'].isna().mean())    # 99.4% of PoolQC records are NA

PoolArea
0      1162
648       1
555       1
480       1
512       1
519       1
738       1
Name: count, dtype: int64
PoolQC
Fa    2
Ex    2
Gd    2
Name: count, dtype: int64
0.9948630136986302


In [ ]:
X_train.pop('PoolQC')
with pd.option_context('display.max_rows', None):
    print(X_train.isna().mean()[X_train.isna().mean() > 0])

In [81]:
X_train['MiscFeature'].value_counts()
X_train['MiscVal'].value_counts()

MiscVal
0        1124
400         8
500         8
700         5
450         4
2000        4
1200        2
480         2
15500       1
600         1
3500        1
8300        1
1300        1
800         1
350         1
620         1
1150        1
1400        1
2500        1
Name: count, dtype: int64

In [107]:
X_train.pop('MiscFeature')
X_train.pop('MiscVal')

254     0
1066    0
638     0
799     0
380     0
       ..
1095    0
1130    0
1294    0
860     0
1126    0
Name: MiscVal, Length: 1168, dtype: int64

In [108]:
with pd.option_context('display.max_rows', None):
    print(X_train.isna().mean()[X_train.isna().mean() > 0])

LotFrontage     0.185788
Alley           0.936644
MasVnrType      0.584760
MasVnrArea      0.005137
BsmtQual        0.023973
BsmtCond        0.023973
BsmtExposure    0.023973
BsmtFinType1    0.023973
BsmtFinType2    0.023973
Electrical      0.000856
FireplaceQu     0.468322
GarageType      0.054795
GarageYrBlt     0.054795
GarageFinish    0.054795
GarageQual      0.054795
GarageCond      0.054795
Fence           0.800514
dtype: float64


In [109]:
X_train['Alley'].value_counts()

Alley
Grvl    44
Pave    30
Name: count, dtype: int64

In [110]:
X_train.pop('Alley')

254      NaN
1066     NaN
638      NaN
799      NaN
380     Pave
        ... 
1095     NaN
1130     NaN
1294     NaN
860      NaN
1126     NaN
Name: Alley, Length: 1168, dtype: object

In [111]:
with pd.option_context('display.max_rows', None):
    print(X_train.isna().mean()[X_train.isna().mean() > 0])

LotFrontage     0.185788
MasVnrType      0.584760
MasVnrArea      0.005137
BsmtQual        0.023973
BsmtCond        0.023973
BsmtExposure    0.023973
BsmtFinType1    0.023973
BsmtFinType2    0.023973
Electrical      0.000856
FireplaceQu     0.468322
GarageType      0.054795
GarageYrBlt     0.054795
GarageFinish    0.054795
GarageQual      0.054795
GarageCond      0.054795
Fence           0.800514
dtype: float64


In [112]:
X_train['Fence'].value_counts()

Fence
MnPrv    128
GdPrv     50
GdWo      46
MnWw       9
Name: count, dtype: int64

In [113]:
X_train['Fence'] = X_train['Fence'].notna()

In [114]:
X_train['Fence'].value_counts()

Fence
False    935
True     233
Name: count, dtype: int64

In [115]:
with pd.option_context('display.max_rows', None):
    print(X_train.isna().mean()[X_train.isna().mean() > 0])

LotFrontage     0.185788
MasVnrType      0.584760
MasVnrArea      0.005137
BsmtQual        0.023973
BsmtCond        0.023973
BsmtExposure    0.023973
BsmtFinType1    0.023973
BsmtFinType2    0.023973
Electrical      0.000856
FireplaceQu     0.468322
GarageType      0.054795
GarageYrBlt     0.054795
GarageFinish    0.054795
GarageQual      0.054795
GarageCond      0.054795
dtype: float64


In [127]:
X_train['MasVnrType'].isna().mean()

np.float64(0.5847602739726028)

In [128]:
X_train.pop('MasVnrType')

254         NaN
1066        NaN
638         NaN
799     BrkFace
380         NaN
         ...   
1095        NaN
1130        NaN
1294        NaN
860         NaN
1126    BrkFace
Name: MasVnrType, Length: 1168, dtype: object

In [129]:
X_train['MasVnrArea'] = X_train['MasVnrArea'].fillna(0)

In [132]:
with pd.option_context('display.max_rows', None):
    print(X_train.isna().mean()[X_train.isna().mean() > 0])

LotFrontage     0.185788
BsmtQual        0.023973
BsmtCond        0.023973
BsmtExposure    0.023973
BsmtFinType1    0.023973
BsmtFinType2    0.023973
Electrical      0.000856
FireplaceQu     0.468322
GarageType      0.054795
GarageYrBlt     0.054795
GarageFinish    0.054795
GarageQual      0.054795
GarageCond      0.054795
dtype: float64
